This notebook uses real valued features and their pca transformed values to create a 40 * 40 image, then pass these images to a 3 convolutional layers and concanate its output vector with output of a linear layer which its inputs are cp_time & cp_dose.
full process of our work is shown in below image:

<img src="https://i.ibb.co/HnK8NDD/cnn.png" alt="cnn" border="0">

In [ ]:
!pip install /kaggle/input/iterate/iterative-stratification-master
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
from  torch.autograd import Variable
from torch.optim import lr_scheduler
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils import data as torch_data 
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, utils
import torchvision 
import torchvision.models as models

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from skimage import io, transform

# from iterate.iterativestratification.iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import itertools
from PIL import Image
import random
import pandas as pd
from tqdm import tqdm
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import logging
import warnings
import gc
warnings.filterwarnings("ignore")
import cv2 as cv
import copy
from random import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import QuantileTransformer

In [ ]:
seed = 2020

cudnn.benchmark = False
cudnn.deterministic = True
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)


In [ ]:
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
train_drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')
sample_submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [ ]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [ ]:
#RankGauss
for col in (GENES + CELLS):

    transformer = QuantileTransformer(n_quantiles=100,random_state=seed, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [ ]:
# import seaborn as sns
# sns.displot(train_features['g-3'].values)

In [ ]:
# GENES
n_comp = 650  #<--Update

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
data2 = (PCA(n_components=n_comp, random_state=seed).fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [ ]:
#CELLS
n_comp = 100  #<--Update

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=seed).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [ ]:
dic1 = {24:1, 72:2, 48:3}
dic2 = {'D1':1, 'D2':2}

train_features['cp_time'] = train_features.cp_time.map(dic1)
train_features['cp_dose'] = train_features.cp_dose.map(dic2)

test_features['cp_time'] = test_features.cp_time.map(dic1)
test_features['cp_dose'] = test_features.cp_dose.map(dic2)

In [ ]:
# bkup_df = train_features.copy()
cols = train_features.columns

In [ ]:
def shuffle_cols(df,cols):
    colnames     = list(cols)
    colnames1    = colnames.copy()
    colnames1    = np.random.RandomState(seed).permutation(colnames1[4:],)
    colnames[4:] = colnames1
    #pd.DataFrame({'columns':colnames}).to_csv('columns.csv',index=False)
    df = df.reindex(columns=colnames)
    return df

In [ ]:
# train_features  = shuffle_cols(train_features,cols)
# test_features   = shuffle_cols(test_features,cols)

In [ ]:
import torch.nn.functional as F
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1)
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1)
        self.conv3 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=1,stride=1)
        self.relu  = nn.ReLU(inplace=True)
        self.bn1   = nn.BatchNorm2d(num_features=32)
        self.bn2   = nn.BatchNorm2d(num_features=64)
        self.bn3   = nn.BatchNorm2d(num_features=128)
        self.mp    = nn.AvgPool2d(kernel_size=2,stride=1)
        self.do    = nn.Dropout2d(p=0.3)
        self.l1    = nn.Linear(139394,700)
        self.l2    = nn.Linear(700, 206)
        self.feat1 = nn.Linear(2,4)
        self.feat2 = nn.Linear(4,2)
        
    def forward(self, x , z):
        x = self.conv_layers1(x)
        x = self.conv_layers2(x)
        x = self.conv_layers3(x)
        N ,_,_,_ = x.size()
        x = x.view(N,-1)
        z = self.feat1(z)
        x = self.do(x)
        z = self.feat2(z)
        
        w = torch.cat((x,z),1)
        w = w.view(N,-1)
        print(w.size())
        w = self.l1(w)
#         x = self.do(x)
        w = self.relu(w)
        w = self.l2(w)
        return w
    
    def conv_layers1(self, x):
        x = self.conv1(x)
        x = self.relu(x)
#         x = self.bn1(x)
        x = self.mp(x)
#         x = self.do(x)
        return x
    
    def conv_layers2(self, x):
        x = self.conv2(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = self.mp(x)
#         x = self.do(x)
        return x
    
    def conv_layers3(self, x):
        x = self.conv3(x)
        x = self.relu(x)
#         x = self.do(x)
#         x = self.bn3(x)
        x = self.mp(x)
        return x


In [ ]:
use_gpu = torch.cuda.is_available()
print(use_gpu)

In [ ]:
weight = train_targets_scored.iloc[:,1:].sum()
wght = 1-(np.log(weight)/np.mean(weight))
wght = wght.values
wght = torch.tensor(wght).cuda()
# wght

In [ ]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = wght
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [ ]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))    

In [ ]:
def convert_to_img(row):
    image = np.array(row[2:].values,dtype=float)
    ax = image[4:1604]
    
    ax= ax.reshape((40,40))
#     ax = np.random.shuffle(ax)
    ax  = (ax - np.min(ax))/(np.max(ax)-np.min(ax))
#     ax = ax * 255
    ax = ax.reshape((-1, ax.shape[0], ax.shape[1]))
    return ax


class myDataset(torch_data.Dataset):
    def __init__(self, feats, labels):
        self.features = feats
        self.labels = labels
        
    def __len__(self):
        return (self.features.shape[0])

    def __getitem__(self, idx):
        img_name = str(self.features.iloc[idx, 0])
        img1 = convert_to_img(self.features.iloc[idx, :])
        feats = self.features.iloc[idx, 2:4]
        d = {"img": img_name}
        d["label"] = torch.tensor(self.labels.iloc[idx, 1:], dtype=torch.float) 
        d["img1"] = torch.tensor(img1, dtype=torch.float)
        d["feats"] = torch.tensor(feats, dtype=torch.float)

        
        return d

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def create_train(X_train,Y_train):
    train_dataset = myDataset(feats = X_train, labels=Y_train)
    trainloader = torch_data.DataLoader(train_dataset, batch_size= 64, shuffle=True)
    return trainloader

def create_test(X_test,Y_test):
    val_dataset   = myDataset(feats = X_test, labels=Y_test)
    valloader = torch_data.DataLoader(val_dataset, batch_size= 64, shuffle=False)
    return valloader
    

In [ ]:
def train(epoches, model,dataloaders,f, num_label=206):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    running_loss_train = []
    running_loss_val = []
    running_corrects_train = []
    running_corrects_val = []
    epoch_acc_hist = {"train": [], "valc": []}
    epoch_loss_hist = {"train": [], "val": []}
    
    
    LEARNING_RATE = 8e-2
    WEIGHT_DECAY = 1e-3
    loss_fn = nn.BCEWithLogitsLoss()
    criterion = SmoothBCEwLogits(smoothing =0.001)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer,
                                              pct_start=0.1,
                                              div_factor=1e3,
                                              max_lr=1e-2, 
                                              epochs=2, 
                                              steps_per_epoch=len(trainloader))
    early_stopping_steps = 3
    early_step = 1
    
    min_loss = 9999999
    print(f"Fold {f}/{n_splt}")
    print("- " * 15)
    for epoch in range(epoches):
        print(f"Epoch {epoch}/{epoches}")

        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0.0
            
            for batch_idx, batch_data in enumerate(tqdm(dataloaders[phase])):
                img1  = Variable(batch_data["img1"].float().cuda())
                feats = Variable(batch_data["feats"].float().cuda())
                label = Variable(batch_data["label"].float().cuda())
                model.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    out = model(img1,feats)
                    batch_loss = 0.0
                    batch_loss += criterion(out, label)

                    if phase == "train":
                        batch_loss.backward()
                        optimizer.step()

                running_loss += batch_loss * img1.size(0)
                if phase == "train":
                    running_loss_train.append(batch_loss.item())
                elif phase == "val":
                    running_loss_val.append(batch_loss.item())
                batch_corrects = 0.0

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_loss_hist[phase].append(epoch_loss)

            print('-------------{} Loss: {:.4f}'.format(phase, epoch_loss))
            
 
                
            if phase == "val" and epoch_loss < min_loss:
                min_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save({"current_model_wts": model.state_dict(),
                            "best_model_wts": best_model_wts,
                            "current_epoch": epoch,
                            "optimizer_wts": optimizer.state_dict(),
                            "running_loss_train": running_loss_train,
                            "running_loss_val": running_loss_val,
                            "epoch_loss_hist": epoch_loss_hist, },
                            "moa_model_fold_{}.pth".format(str(f)))
        if epoch_loss >= min_loss:
            early_step = early_step + 1
        if early_step == early_stopping_steps:
            break
                
    gc.collect()
    time_elapsed = time.time() - since
    print('Fold Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Min val Loss: {:4f}'.format(min_loss))


In [ ]:
n_splt = 6
mskf = MultilabelStratifiedKFold(n_splits=n_splt)
epoches = 5
model = None

for f, (t_idx, v_idx) in enumerate(mskf.split(X=train_features, y=train_targets_scored)):
    train_df = train_features.loc[t_idx,:].reset_index(drop=True)
    train_y_df = train_targets_scored.loc[t_idx,:].reset_index(drop=True)
    
    valid_df = train_features.loc[v_idx,:].reset_index(drop=True)
    valid_y_df = train_targets_scored.loc[v_idx,:].reset_index(drop=True)
    
    trainloader = create_train(train_df,train_y_df)
    validloader = create_train(valid_df,valid_y_df)
    
    dataloaders = {"train": trainloader, "val": validloader}
#     dataset_sizes = {"train":  len(train_dataset), "val": len(val_dataset)}
    if model != None:
        del model
    torch.cuda.empty_cache()

    model = Model()
    if use_gpu:
        model = model.cuda()
        
    train(epoches, model, dataloaders,f, num_label = 206)
    
    
    

In [ ]:
def inference_fn(model, dataloader):
    model.eval()
    preds  = []
    labels = []
    
    for batch_idx, batch_data in enumerate(tqdm(dataloader)):
        img1  = Variable(batch_data["img1"].float().cuda())
        feats = Variable(batch_data["feats"].float().cuda())
        label = Variable(batch_data["label"].float().cuda())
#         inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(img1,feats)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        labels.append(label.cpu().numpy())
        
    preds = np.concatenate(preds)
    labels = np.concatenate(labels)
    return preds,labels

In [ ]:
validloader = create_train(test_features,sample_submission)

In [ ]:
for i in range(n_splt):
    if model != None:
        del model
    torch.cuda.empty_cache()
    
    model = Model()
    model.eval()
    model.load_state_dict(torch.load(f"moa_model_fold_{i}.pth")['current_model_wts'])
    

    model.to(device)

    preds,labels = inference_fn(model, validloader)
    preds += preds
submission = preds/n_splt

In [ ]:
preds

In [ ]:
# preds = preds - (np.mean(preds)/3)
preds[preds<0] = 0.000123
sample_submission.iloc[:,1:] = preds
sample_submission = sample_submission.fillna(0)
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(100)